In [22]:
# Import required libraries
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19, ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [4]:
image_size = (128, 128)
batch_size = 32
validation_split = 0.2
num_classes = 15

dataset_dir = r"D:\Ashraf\NCI\Semester 2\DMML2\CA2\Dataset\PlantVillage"
base_dir = r"D:\Ashraf\NCI\Semester 2\DMML2\CA2\Dataset\New"

# Function to create directories
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Split dataset into training and validation sets
def split_dataset(dataset_dir, base_dir, validation_split):
    create_dir(base_dir)
    train_dir = os.path.join(base_dir, 'train')
    validation_dir = os.path.join(base_dir, 'validation')
    create_dir(train_dir)
    create_dir(validation_dir)

    for category in os.listdir(dataset_dir):
        category_path = os.path.join(dataset_dir, category)
        if os.path.isdir(category_path):
            images = os.listdir(category_path)
            train_images, val_images = train_test_split(images, test_size=validation_split, random_state=42)

            train_category_dir = os.path.join(train_dir, category)
            val_category_dir = os.path.join(validation_dir, category)
            create_dir(train_category_dir)
            create_dir(val_category_dir)

            for image in train_images:
                shutil.copy(os.path.join(category_path, image), os.path.join(train_category_dir, image))

            for image in val_images:
                shutil.copy(os.path.join(category_path, image), os.path.join(val_category_dir, image))

split_dataset(dataset_dir, base_dir, validation_split)

In [27]:
# Data Preparation
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

validation_dataset = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

Found 16504 images belonging to 15 classes.
Found 4134 images belonging to 15 classes.


# VGG19

In [28]:
# Build VGG19 Model
vgg_base = VGG19(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
for layer in vgg_base.layers:
    layer.trainable = False
x_vgg = GlobalAveragePooling2D()(vgg_base.output)
x_vgg = Dense(1024, activation='relu')(x_vgg)
predictions_vgg = Dense(num_classes, activation='softmax')(x_vgg)
vgg_model = Model(inputs=vgg_base.input, outputs=predictions_vgg)

In [29]:
# Compile the Model
vgg_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
# Train the Model
num_epochs = 5

vgg_history = vgg_model.fit(
    train_dataset,
    steps_per_epoch=train_dataset.samples // batch_size,
    epochs=num_epochs,
    validation_data=validation_dataset,
    validation_steps=validation_dataset.samples // batch_size)

Epoch 1/5


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


515/515 ━━━━━━━━━━━━━━━━━━━━ 1617s 3s/step - accuracy: 0.4871 - loss: 1.6346 - val_accuracy: 0.7025 - val_loss: 0.8891
Epoch 2/5
  1/515 ━━━━━━━━━━━━━━━━━━━━ 21:56 3s/step - accuracy: 0.7188 - loss: 0.9927

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7188 - loss: 0.9927 - val_accuracy: 0.8333 - val_loss: 0.5483
Epoch 3/5
515/515 ━━━━━━━━━━━━━━━━━━━━ 1605s 3s/step - accuracy: 0.6901 - loss: 0.9340 - val_accuracy: 0.7425 - val_loss: 0.7526
Epoch 4/5
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 986us/step - accuracy: 0.7500 - loss: 0.6208 - val_accuracy: 0.8333 - val_loss: 0.5169
Epoch 5/5
515/515 ━━━━━━━━━━━━━━━━━━━━ 1351s 3s/step - accuracy: 0.7165 - loss: 0.8319 - val_accuracy: 0.7590 - val_loss: 0.6939


In [31]:
# Evaluation of VGG Model
vgg_loss, vgg_accuracy = vgg_model.evaluate(validation_dataset)

print(f"VGG Validation accuracy: {vgg_accuracy*100:.2f}%")

130/130 ━━━━━━━━━━━━━━━━━━━━ 175s 1s/step - accuracy: 0.7666 - loss: 0.6823
VGG Validation accuracy: 75.88%


# Plant Care

In [43]:
# Disease information dictionary
disease_info = {
    'Pepper__bell___Bacterial_spot': {
        'sunlight': '6-8 hours of direct sunlight',
        'water': 'Keep soil consistently moist but not waterlogged',
        'water_liters': '1-1.5 liters per day',
        'cure': 'Remove infected leaves and apply copper-based fungicides'
    },
    'Pepper__bell___healthy': {
        'sunlight': '6-8 hours of direct sunlight',
        'water': 'Water when the top 1 inch of soil is dry',
        'water_liters': '1 liter per day',
        'cure': 'No disease detected, maintain healthy growth practices'
    },
    'Potato___Early_blight': {
        'sunlight': 'Full sun',
        'water': 'Water deeply, allowing soil to dry between waterings',
        'water_liters': '1-1.5 liters per day',
        'cure': 'Apply fungicides and remove affected leaves'
    },
    'Potato___Late_blight': {
        'sunlight': 'Full sun',
        'water': 'Keep soil evenly moist but avoid overwatering',
        'water_liters': '1-1.5 liters per day',
        'cure': 'Use resistant varieties and apply appropriate fungicides'
    },
    'Potato___healthy': {
        'sunlight': 'Full sun',
        'water': 'Water when the top 2 inches of soil are dry',
        'water_liters': '1-1.5 liters per day',
        'cure': 'No disease detected, maintain healthy growth practices'
    },
    'Tomato_Bacterial_spot': {
        'sunlight': '6-8 hours of direct sunlight',
        'water': 'Water at the base, avoiding wetting the foliage',
        'water_liters': '1-1.5 liters per day',
        'cure': 'Remove infected leaves and apply copper-based fungicides'
    },
    'Tomato_Early_blight': {
        'sunlight': '6-8 hours of direct sunlight',
        'water': 'Water deeply, allowing soil to dry between waterings',
        'water_liters': '1.5-2 liters per day',
        'cure': 'Apply fungicides and remove affected leaves'
    },
    'Tomato_Late_blight': {
        'sunlight': '6-8 hours of direct sunlight',
        'water': 'Keep soil evenly moist but avoid overwatering',
        'water_liters': '1.5-2 liters per day',
        'cure': 'Use resistant varieties and apply appropriate fungicides'
    },
    'Tomato_Leaf_Mold': {
        'sunlight': '6-8 hours of direct sunlight',
        'water': 'Water at the base, keeping foliage dry',
        'water_liters': '1.5-2 liters per day',
        'cure': 'Improve air circulation and apply fungicides'
    },
    'Tomato_Septoria_leaf_spot': {
        'sunlight': '6-8 hours of direct sunlight',
        'water': 'Water at the base, avoiding wetting the foliage',
        'water_liters': '1.5-2 liters per day',
        'cure': 'Remove infected leaves and apply fungicides'
    },
    'Tomato_Spider_mites_Two_spotted_spider_mite': {
        'sunlight': '6-8 hours of direct sunlight',
        'water': 'Maintain consistent soil moisture',
        'water_liters': '1-1.5 liters per day',
        'cure': 'Use insecticidal soaps or miticides'
    },
    'Tomato__Target_Spot': {
        'sunlight': '6-8 hours of direct sunlight',
        'water': 'Water at the base, avoiding wetting the foliage',
        'water_liters': '1.5-2 liters per day',
        'cure': 'Remove infected leaves and apply fungicides'
    },
    'Tomato__Tomato_YellowLeaf__Curl_Virus': {
        'sunlight': '6-8 hours of direct sunlight',
        'water': 'Water regularly, keeping soil evenly moist',
        'water_liters': '1.5-2 liters per day',
        'cure': 'Remove affected plants and control whitefly population'
    },
    'Tomato__Tomato_mosaic_virus': {
        'sunlight': '6-8 hours of direct sunlight',
        'water': 'Keep soil consistently moist',
        'water_liters': '1-1.5 liters per day',
        'cure': 'Remove and destroy infected plants, practice crop rotation'
    },
    'Tomato_healthy': {
        'sunlight': '6-8 hours of direct sunlight',
        'water': 'Water when the top 1 inch of soil is dry',
        'water_liters': '1.5 liters per day',
        'cure': 'No disease detected, maintain healthy growth practices'
    }
}


In [44]:
# Function to get care recommendations based on predicted class
def get_care_recommendations(predicted_class):
    if predicted_class in disease_info:
        return disease_info[predicted_class]
    else:
        return {
            'sunlight': 'Information not available',
            'water': 'Information not available',
            'water_liters': 'Information not available',
            'cure': 'Information not available'
        }

In [47]:
class_labels = list(validation_dataset.class_indices.keys())
print("Class labels from validation generator:", class_labels)


Class labels from validation generator: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [48]:
# Predict and provide care recommendations for the first image in the validation set
validation_dataset.reset()
vgg_predictions = vgg_model.predict(validation_dataset)
vgg_predicted_classes = np.argmax(vgg_predictions, axis=1)

# Get recommendations for the first predicted class
predicted_class = class_labels[vgg_predicted_classes[0]]
recommendations = get_care_recommendations(predicted_class)
print(f"Recommendations for {predicted_class}:")
print(f"Sunlight: {recommendations['sunlight']}")
print(f"Water: {recommendations['water']}")
print(f"Water in liters: {recommendations['water_liters']}")
print(f"Cure: {recommendations['cure']}")


130/130 ━━━━━━━━━━━━━━━━━━━━ 215s 2s/step
Recommendations for Tomato_Bacterial_spot:
Sunlight: 6-8 hours of direct sunlight
Water: Water at the base, avoiding wetting the foliage
Water in liters: 1-1.5 liters per day
Cure: Remove infected leaves and apply copper-based fungicides
